In [1]:
import numpy as np
from scipy.special import gammaln
import random
from collections import Counter
import pickle
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import pickle
import graphviz
import pydot
import pygraphviz

In [2]:
%load_ext Cython

The example we are going to use. We just set the simplest scenarios

In [3]:
topic=[[2, 2, 2, 3], [0, 1, 2, 3, 3, 3, 3], [], [1, 1]]

corpus=[['a'],
 ['b', 'c', 'd'],
 ['e', 'f', 'a', 'c'],
 ['g', 'd', 'e', 'b', 'a'],
 ['a'],
 ['b', 'c', 'd'],
 ['e', 'f', 'a', 'c'],
 ['g', 'd', 'e', 'b', 'a']]

#### wrap all of all codes into  *cython*

In [5]:
%%cython --annotate

cimport cython
import cython
cimport numpy as np
cimport cython
import cython
cimport numpy as np

import numpy as np

import math
from scipy.special import gammaln
import numpy as np
from collections import Counter

@cython.cdivision
@cython.boundscheck(False)
@cython.wraparound(False)


cdef Z(list corpus, int T, double alpha, double beta):
    """
    Description
    ---------
    Funcion:  sample zmn under LDA model
    
    Parameter
    ---------
    corpus: the total corpus, a list of documents, that is, a list of lists
    T: the number of topics
    alpha, beta: parameters
    
    Return
    ------
    topic: the word list in each topic
    topic_num: the length of each topic
    """
    cdef int W = np.sum([len(word) for word in corpus]) 
    cdef int N = len(corpus)
    cdef list topic = [[] for t in range(T)]
    cdef list topic_num=[[] for t in range(T)]
    
    cdef int i
    cdef int j
    cdef list di
    cdef str wi
    cdef double[:] p
    cdef double[:] pp=np.zeros(T)

    for i,di in enumerate(corpus):
        for wi in di:
            p=np.zeros(T)
            for j in range(T):                
                p[j]=((topic[j].count(wi)+beta)/(len(topic[j])+W*beta)) * ((np.sum(np.isin(topic[j],di))+alpha)/(len(di)+T*alpha))
            for j in range(T): 
                pp[j]=p[j]/np.sum(p)
            topic[int(np.where(np.random.multinomial(1, pp, size=1)[0]==1)[0])].append(wi)
            topic_num[int(np.where(np.random.multinomial(1, pp, size=1)[0]==1)[0])].append(i)
    return(topic,topic_num)


cdef CRP_next(double lambdas, list topic):
  
    cdef int N
    cdef list word_list  = []
    cdef list t
    cdef list x
    cdef double m
    cdef int i
    cdef double[:] tables 
    cdef double[:] p_old
    cdef double p_new
    cdef list p
    cdef double[:] P
    
    N=len(topic) 
    
    for t in topic:
        word_list=word_list+t
    
    m=len(word_list) 
    p_old = np.zeros(N)
    for (i,x) in enumerate(topic):
        p_old[i]=len(x)/(lambdas+m) 
    p_new=lambdas/(lambdas+m)      
    p=[p_new]+list(p_old) 
    
    return(p)

cdef topics(list corpus,double lambdas):
    cdef list topic = [] 
    cdef list docs
    cdef str word  
    cdef int position
    
    for docs in corpus:
        for word in docs:
            p = CRP_next(lambdas,topic)
            position=int(np.where((np.random.multinomial(1,list((np.array(p)/sum(p)))))!=0)[0])
            if position==0:
                topic.append([word])
            else:
                topic[position-1].append(word)
    return(topic)

cdef CRP_prior(list corpus,list topic,double lambdas):
    cdef list res
    cdef int i
    cdef int j
    cdef list docus
    cdef list p_topic
    cdef list temp
    cdef list x
    
    res=[]
    for i,docs in enumerate(corpus):
        p_topic=[]
        for j in range(len(topic)):
            temp=[]
            for k in range(len(topic[j])):
                if topic[j][k] != i:
                    temp.append(topic[j][k])
            p_topic.append(temp)
        temp = CRP_next(lambdas,p_topic)[1:]
        res.extend(temp)
    return(np.array(res).reshape(len(corpus),len(topic)))


cdef word_likelihood(list corpus,list topic,double eta):
    cdef double[:,:] res=np.zeros((len(corpus),len(topic)))
    cdef list word_list=[] 
    cdef int W
    cdef double part1_denominator=1
    cdef double part2_nominator=1 
    cdef double part1_nominator
    cdef double part2_denominator
    cdef int i
    cdef int j
    cdef list di
    cdef double p_w
    cdef double ncm_w
    cdef double nc_w
    cdef str word
    
    for i in range(len(corpus)):
        word_list=word_list+corpus[i]
    W=len(word_list)                        # the length of word list
    
    for i,di in enumerate(corpus):
        p_w=0
        for j in range(len(topic)):         
            nc_dot=len(topic[j])                
            part1_nominator = gammaln(len(topic[j])-(len(set(topic[j]))-len(set(topic[j])-set(di)))+W*eta)
            part2_denominator = gammaln(len(topic[j])+W*eta)
        
            for word in di:
                ncm_w=topic[j].count(word)-di.count(word)
                if ncm_w <0:
                    ncm_w=0
                nc_w=topic[j].count(word)
                part1_denominator=part1_denominator+gammaln(ncm_w+eta)
                part2_nominator=part2_nominator+gammaln(nc_w+eta)
            
            p_w=part1_nominator-part1_denominator+part2_nominator-part2_denominator 
            res[i,j]=p_w
        for j in range(len(topic)): 
            res[i,j] = res[i,j] + np.abs(np.min(res[i, :]) + 0.1)
            
    for i in range(len(corpus)):
        for j in range(len(topic)):   
            res[i,j]=res[i,j]/np.sum(res[i,:])
            
    return(np.asarray(res))

cdef nodes(list corpus,int T, double alpha, double beta,double lambdas, double eta,int iters):
    
    cdef list word_list
    cdef list x
    cdef int W

    cdef list topic
  #  cdef double[:,:] c_

    word_list=[]
    for x in corpus:
        word_list=word_list+x
    W=len(word_list)
    gibbs=np.zeros((W,iters))
    
    for j in range(iters):
       # print('iters % j complete' ,j)
        topic=Z(corpus, T, alpha, beta)[1]
        w_m=word_likelihood(corpus,topic,eta)
        c_=CRP_prior(corpus,topic,lambdas)
        c_m = (w_m * c_) / (w_m * c_).sum(axis = 1).reshape(-1,1)
        
        g=[]
        for i,docs in enumerate(corpus):
            if np.sum(c_m[i,:-1])>1:
                c_m[i,-1]=0
                c_m[i,:-1]=c_m[i,:-1]/np.sum(c_m[i,:-1])
            for word in docs:     
                g.append(int(np.where(np.random.multinomial(1, c_m[i])!=0)[0]))
        gibbs[:,j]=g
        
    word_topic=[]
    for i in range(W):
        word_topic.append(int(Counter(gibbs[i]).most_common(1)[0][0]))
    n_topic=np.max(word_topic)+1
    
    wn_topic = [[] for _ in range(n_topic)]
    wn_doc_topic = [[] for _ in range(n_topic)]

    n = 0
    for i in range(len(corpus)):
        for word in corpus[i]:
            #print(n)
            wn_doc_topic[word_topic[n]].append(word)
            n=n+1
        for j in range(n_topic):
            if wn_doc_topic[j] != []:
                wn_topic[j].append(wn_doc_topic[j])
        wn_doc_topic = [[] for _ in range(n_topic)]        

    wn_topic = [x for x in wn_topic if x != []]
    
    return(wn_topic) 

def chierarchical_LDA(list corpus,double alpha,double beta,double lambdas,double eta,int iters,int level, int num):
    
    cdef int i
    cdef int j
    
    topic = topics(corpus, lambdas)    
    node = [[] for _ in range(level)]
    node_num = [[] for _ in range(level+1)]
    node_num[0].append(1)
    
    word_topic = nodes(corpus=corpus, T=len(topic), alpha=alpha, beta=beta, lambdas=lambdas, eta=eta, iters=iters)
    words = sum(word_topic[0],[])
    node[0].append(words)
    print_word=list(dict(Counter(words).most_common(num)).keys())
   
    temp=word_topic[1:]
    node_num[1].append(len(word_topic[1:]))
    
    for i in range(1,level):
        
        for j in range(sum(node_num[i])):
            if len(temp)<1:
                break
            word_topic2 = nodes(corpus=temp[0],  T=len(topic), alpha=alpha, beta=beta, lambdas=lambdas, eta=eta, iters=iters)
            words2 = sum(word_topic2[0],[])
            node[i].append(words2)
            print_word2=list(dict(Counter(words2).most_common(num)).keys())
        
            temp=temp[1:]
            if len(word_topic2)>2:
                temp.extend(word_topic2[1:])
            node_num[i+1].append(len(word_topic2[1:]))
    return(node,node_num[:level])

#### Read in codes without optimization

In [48]:
import hierarchical_LDA

#### Compare the time

without optimization

In [49]:
np.random.seed(123)

%timeit -r1 -n1  hierarchical_LDA.hierarchical_LDA(corpus, 0.1, 0.1,1,1,100,3,5)

1.29 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


with optimization, quicker!

In [50]:
np.random.seed(123)

%timeit -r1 -n1  chierarchical_LDA(corpus, 0.1, 0.1,1,1,100,3,5)

738 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
